In [1]:
from flask import Flask,request

In [2]:
from flask_restful import Resource,Api

In [3]:
class User:
    def __init__(self,_id,username,password):
        self.id=_id
        self.username = username
        self.password =password

In [4]:
users=[
   User(1,'faiz','abc')
]
username_mapping ={u.username:u for u in users}
userid_mapping = {u.id:u for u in users}
def authenticate(username,password):
    user = username_mapping.get(username,None)
    if user and user.password == password:
        return user
def identity(payload):
    user_id = payload['identity']
    return userid_mapping.get(user_id,None)

In [ ]:
from werkzeug.wrappers import Request, Response
from flask_jwt import JWT,jwt_required

app = Flask(__name__)
app.secret_key = "faiz"
api=Api(app)

jwt = JWT(app,authenticate,identity)

items=[]
class Item(Resource):
    @jwt_required()
    def get(self,name):
        item = next(filter(lambda x:x['name']==name,items),None)
        return {"item":item},200 if item else 404
    def post(self,name):
        if next(filter(lambda x:x['name']==name,items),None):
           return {'message':"An item with name '{}' already exists".format(name)},400 
        data=request.get_json()
        item={"name":name,"price":data['price']}
        items.append(item)
        return item,201
    def delete(self,name):
        global items
        items=list(filter(lambda x:x['name']!=name,items))
        return {"message":"Item deleted"}
class ItemList(Resource):
    def get(self):
        return {"items":items}
api.add_resource(Item,'/item/<string:name>')
api.add_resource(ItemList,'/items')
if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple('localhost', 9000, app)

 * Running on http://localhost:9000/ (Press CTRL+C to quit)
127.0.0.1 - - [17/Jul/2018 21:04:52] "POST /auth HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2018 21:05:02] "POST /auth HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2018 21:05:15] "POST /item/tv HTTP/1.1" 201 -
127.0.0.1 - - [17/Jul/2018 21:05:30] "POST /auth HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2018 21:06:18] "POST /items HTTP/1.1" 405 -
127.0.0.1 - - [17/Jul/2018 21:06:33] "GET /items HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2018 21:06:43] "DELETE /item/tv HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2018 21:06:47] "GET /items HTTP/1.1" 200 -
